In [62]:
# pip install dydx-v3-python
# pip install python-dotenv
# pip install web3

from dydx3 import Client
from web3 import Web3
import os
import time
from dotenv import load_dotenv



In [72]:
# set up client and variables

load_dotenv()
# variables in .env need to be = WALLETADDRESS, INFURAENDPOINT, ETHPRIVATEKEY

walletAddress = os.getenv("WALLETADDRESS")
ethPrivateKey = os.getenv("ETHPRIVATEKEY")
infuraEndpoint = os.getenv("INFURAENDPOINT")


public_client = Client(
    host='https://api.dydx.exchange',
    default_ethereum_address= walletAddress,
)




In [64]:
# only do this if you want to trade rather than query
private_client = Client(
    host='https://api.dydx.exchange',
    default_ethereum_address= walletAddress,
    eth_private_key= ethPrivateKey,
    web3=Web3(Web3.HTTPProvider(infuraEndpoint)),
    # stark_private_key='01234abcd...',
)
stark_key = private_client.onboarding.derive_stark_key(walletAddress,)
api_key = private_client.onboarding.recover_default_api_key_credentials(walletAddress)
private_client_with_key = Client(
    host='https://api.dydx.exchange',
    default_ethereum_address= walletAddress,
    api_key_credentials=api_key,
    stark_private_key=stark_key["private_key"]
)


In [65]:
# main functions
def getWtAvePriceDyDx3(size, orderbook):
    sizeLeft = float(size)
    wtAvePx = 0
    for entry in orderbook:
        if sizeLeft <= 0:
            break
        sizeUsed = min(sizeLeft, float(entry["size"]))
        wtAvePx += float(entry["price"]) * sizeUsed
        sizeLeft -= sizeUsed
    return (wtAvePx / (size - sizeLeft)) # just in case the order is bigger than the market depth

def getBidOfferForSizeDyDx3(size, orderbook):
    if (size <= 0):
        size = 1
    return (getWtAvePriceDyDx3(size, orderbook["bids"]), getWtAvePriceDyDx3(size, orderbook["asks"]))


In [71]:

# allmarkets = public_client.public.get_markets().data
# print(allmarkets["markets"]["ETH-USD"])  # https://dydxprotocol.github.io/v3-teacher/?python#market

# orderbook = public_client.public.get_orderbook(market="ETH-USD",).data # get orderbook (e.g. ETH-USD)
# print(getBidOfferForSizeDyDx3(10,orderbook)) # get the bid and offer in 10

# acct = private_client_with_key.private.get_account(walletAddress).data
# print(acct['account']['freeCollateral']) # get available balance of margin in USD 

# private_client_with_key.private.get_orders(market="SOL-USD", status="OPEN").data) # get open orders
# private_client_with_key.private.cancel_order(order_id="14ac73b7c2aaacdae3c40aff54019936ca1fa3137b3de81ec605d9ff989e1c9") # cancel a specific order

# place a limit order
# oneMonthInSeconds = 60*60*24*30
# oneMonthFromNow = int(time.time() + oneMonthInSeconds)
# acct = private_client_with_key.private.get_account(walletAddress).data
# positionID = (int(acct["account"]["positionId"]))
# res = private_client_with_key.private.create_order(position_id=positionID, market= "SOL-USD", side="BUY", order_type="LIMIT", post_only=False, size="20", price="101", limit_fee="0.015", expiration_epoch_seconds=oneMonthFromNow)
